<a href="https://colab.research.google.com/github/Sadath43/Projet-Innovation-groupe-13/blob/main/code_%C3%A0_besoin_de_cr%C3%A9dit_de_consommation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import seaborn as sns
from matplotlib import pyplot as plt

In [32]:
train=pd.read_csv('train.csv')

séléction des features

In [33]:
train=train.drop(["ID","day","month","duration"],axis=1)

In [34]:
# il n'y a pas de nan (valeur manquante)
train.isna().sum() 

age           0
job           0
marital       0
education     0
default       0
balance       0
housing       0
loan          0
contact       0
campaign      0
pdays         0
previous      0
poutcome      0
subscribed    0
dtype: int64

In [35]:
# il faut seulement valeur numérique, donc il faut des transformations
ohe = OneHotEncoder(sparse = False)

train = pd.concat((train , pd.DataFrame(ohe.fit_transform(train["job"].to_frame()),columns = "job_" + np.sort(train["job"].unique()))),axis = 1)
train.drop(columns = ["job"],inplace = True)

train = pd.concat((train , pd.DataFrame(ohe.fit_transform(train["marital"].to_frame()),columns = "marital_" + np.sort(train["marital"].unique()))),axis = 1)
train.drop(columns = ["marital"],inplace = True)

In [36]:
#education
train.loc[(train.education == "tertiary"),"education"] =3
train.loc[(train.education == "secondary") ,"education"] =2
train.loc[(train.education == "primary"),"education"] =1
train.loc[(train.education == "unknown"),"education"] =0
#Default 
train.loc[(train.default == "yes"),"default"] = 1
train.loc[(train.default == "no") ,"default"] = 0

In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31647 entries, 0 to 31646
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                31647 non-null  int64  
 1   education          31647 non-null  object 
 2   default            31647 non-null  object 
 3   balance            31647 non-null  int64  
 4   housing            31647 non-null  object 
 5   loan               31647 non-null  object 
 6   contact            31647 non-null  object 
 7   campaign           31647 non-null  int64  
 8   pdays              31647 non-null  int64  
 9   previous           31647 non-null  int64  
 10  poutcome           31647 non-null  object 
 11  subscribed         31647 non-null  object 
 12  job_admin.         31647 non-null  float64
 13  job_blue-collar    31647 non-null  float64
 14  job_entrepreneur   31647 non-null  float64
 15  job_housemaid      31647 non-null  float64
 16  job_management     316

In [38]:
# housing
train.loc[(train.housing == "yes"),"housing"] = 1 
train.loc[(train.housing == "no") ,"housing"] = 0
#Loan 
train.loc[(train.loan == "yes"),"loan"] = 1
train.loc[(train.loan == "no") ,"loan"] = 0
#contact
train.loc[(train.contact == "telephone"),"contact"] = 2
train.loc[(train.contact == "cellular") ,"contact"] = 1
train.loc[(train.contact == "unknown") ,"contact"] = 0

In [39]:
#poutcome
train = pd.concat((train , pd.DataFrame(ohe.fit_transform(train["poutcome"].to_frame()),columns = "poutcome_" + np.sort(train["poutcome"].unique()))),axis = 1)
train.drop(columns = ["poutcome"],inplace = True)
#subscribed
train.loc[(train.subscribed == "yes"),"subscribed"] = 1 
train.loc[(train.subscribed == "no") ,"subscribed"] = 0

In [40]:
#vérification
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31647 entries, 0 to 31646
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                31647 non-null  int64  
 1   education          31647 non-null  object 
 2   default            31647 non-null  object 
 3   balance            31647 non-null  int64  
 4   housing            31647 non-null  object 
 5   loan               31647 non-null  object 
 6   contact            31647 non-null  object 
 7   campaign           31647 non-null  int64  
 8   pdays              31647 non-null  int64  
 9   previous           31647 non-null  int64  
 10  subscribed         31647 non-null  object 
 11  job_admin.         31647 non-null  float64
 12  job_blue-collar    31647 non-null  float64
 13  job_entrepreneur   31647 non-null  float64
 14  job_housemaid      31647 non-null  float64
 15  job_management     31647 non-null  float64
 16  job_retired        316

In [41]:
train['education'].value_counts()

2    16224
3     9301
1     4808
0     1314
Name: education, dtype: int64

In [42]:
# transformation
train.education = train.education.astype(int)
train.default = train.default.astype(int)
train.housing = train.housing.astype(int)
train.loan = train.loan.astype(int)
train.contact = train.contact.astype(int)
train.subscribed = train.subscribed.astype(int)

In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31647 entries, 0 to 31646
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                31647 non-null  int64  
 1   education          31647 non-null  int64  
 2   default            31647 non-null  int64  
 3   balance            31647 non-null  int64  
 4   housing            31647 non-null  int64  
 5   loan               31647 non-null  int64  
 6   contact            31647 non-null  int64  
 7   campaign           31647 non-null  int64  
 8   pdays              31647 non-null  int64  
 9   previous           31647 non-null  int64  
 10  subscribed         31647 non-null  int64  
 11  job_admin.         31647 non-null  float64
 12  job_blue-collar    31647 non-null  float64
 13  job_entrepreneur   31647 non-null  float64
 14  job_housemaid      31647 non-null  float64
 15  job_management     31647 non-null  float64
 16  job_retired        316

In [44]:
sm = SMOTE()
y = train["loan"]
X = train.drop(columns=["loan"])
y.value_counts()

0    26516
1     5131
Name: loan, dtype: int64

In [45]:
X, y = sm.fit_resample(X, y)
X_train , X_test , y_train , y_test = train_test_split(X,y, test_size = 0.2,random_state = 10)
y.value_counts()

0    26516
1    26516
Name: loan, dtype: int64

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53032 entries, 0 to 53031
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                53032 non-null  int64  
 1   education          53032 non-null  int64  
 2   default            53032 non-null  int64  
 3   balance            53032 non-null  int64  
 4   housing            53032 non-null  int64  
 5   contact            53032 non-null  int64  
 6   campaign           53032 non-null  int64  
 7   pdays              53032 non-null  int64  
 8   previous           53032 non-null  int64  
 9   subscribed         53032 non-null  int64  
 10  job_admin.         53032 non-null  float64
 11  job_blue-collar    53032 non-null  float64
 12  job_entrepreneur   53032 non-null  float64
 13  job_housemaid      53032 non-null  float64
 14  job_management     53032 non-null  float64
 15  job_retired        53032 non-null  float64
 16  job_self-employed  530

In [47]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
matrice_f_knn = confusion_matrix(y_test, y_pred_knn)
performance_knn = accuracy_score(y_test, y_pred_knn)

In [48]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred_lr = lr.predict(X_test)
matrice_f_lr = confusion_matrix(y_test, y_pred_lr)
performance_lr = accuracy_score(y_test, y_pred_lr)


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [49]:
dtc = DecisionTreeClassifier(max_depth=3)
dtc.fit(X_train, y_train)
y_pred_dtc = dtc.predict(X_test)
matrice_f_dtc = confusion_matrix(y_test, y_pred_dtc)
performance_dtc = accuracy_score(y_test, y_pred_dtc)

In [50]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
matrice_f_rf = confusion_matrix(y_test,y_pred_rf)
performance_rf = accuracy_score(y_test,y_pred_rf)

In [51]:
#1-KNN : KNeighborsClassifier (K-Nearest Neighbor)
matrice_f_knn,performance_knn,precision_score(y_test,y_pred_knn),recall_score(y_test,y_pred_knn)

(array([[3172, 2100],
        [ 609, 4726]]),
 0.7446026209107194,
 0.6923527688250806,
 0.8858481724461106)

In [52]:
#2-reggression logistique
matrice_f_lr,performance_lr,precision_score(y_test,y_pred_lr),recall_score(y_test,y_pred_lr)

(array([[3465, 1807],
        [2107, 3228]]),
 0.6309983972848119,
 0.6411122144985104,
 0.6050609184629803)

In [53]:
#3-arbre de décision
matrice_f_dtc,performance_dtc,precision_score(y_test,y_pred_dtc),recall_score(y_test,y_pred_dtc)

(array([[5211,   61],
        [3394, 1941]]),
 0.6742717073630621,
 0.9695304695304695,
 0.36382380506091844)

In [54]:
#4-classificateur random forest
matrice_f_rf,performance_rf,precision_score(y_test,y_pred_rf),recall_score(y_test,y_pred_rf)

(array([[5018,  254],
        [ 967, 4368]]),
 0.8848873385500141,
 0.9450454348766768,
 0.8187441424554827)

In [55]:
def A_besoin_de_credit_de_consommation(model,nom='il',age=56,education=0,default=0,balance=1933,housing=0,loan=0,contact=2,campaign=2,pdays=-1,previous=0,job_admin=1,job_bluecollar=0,job_entrepreneur=0,job_housemaid=0,job_management=0,job_retired=0,job_selfemployed=0,job_services=0,job_student=0,job_technician=0,job_unemployed=0,job_unknown=0,marital_divorced=1,marital_married=0,marital_single=0,poutcome_failure=0,poutcome_other=0,poutcome_success=0,poutcome_unknown=1):
    x=np.array([age,education,default,balance,housing,loan,contact,campaign,pdays,previous,job_admin,job_bluecollar,job_entrepreneur,job_housemaid,job_management,job_retired,job_selfemployed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown]).reshape(1,29)
    predicted=model.predict(x)
    proba_oui=model.predict_proba(x)[0][1]*100
    proba_non=model.predict_proba(x)[0][0]*100
    if predicted[0]==1:
        print("il y a",proba_oui,"% de chance que",nom,"ait besoin d'un credit de consommation")
    else:print("il y a",proba_non,"% de chance que",nom,"ait besoin d'un credit de consommation")

In [56]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53032 entries, 0 to 53031
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                53032 non-null  int64  
 1   education          53032 non-null  int64  
 2   default            53032 non-null  int64  
 3   balance            53032 non-null  int64  
 4   housing            53032 non-null  int64  
 5   contact            53032 non-null  int64  
 6   campaign           53032 non-null  int64  
 7   pdays              53032 non-null  int64  
 8   previous           53032 non-null  int64  
 9   subscribed         53032 non-null  int64  
 10  job_admin.         53032 non-null  float64
 11  job_blue-collar    53032 non-null  float64
 12  job_entrepreneur   53032 non-null  float64
 13  job_housemaid      53032 non-null  float64
 14  job_management     53032 non-null  float64
 15  job_retired        53032 non-null  float64
 16  job_self-employed  530

In [57]:
#A_besoin_de_credit_de_consommation(rf,'Joh',0 ,1,2, 3  ,4,5,6,7, 8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28)
A_besoin_de_credit_de_consommation(rf,'John',20,0,0,1933,0,0,2,2,-1,0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1)

il y a 92.0 % de chance que John ait besoin d'un credit de consommation


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
